In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# ==========================================
# 1. 환경 설정 (수정할 부분)
# ==========================================
DATA_PATH = "./dataset"   # 데이터가 들어있는 최상위 폴더
MAX_FRAMES = 100          # 영상 하나당 최대 길이 (프레임 수)
IMG_WIDTH = 1920          # 좌표 정규화를 위한 영상 가로 크기
IMG_HEIGHT = 1080         # 좌표 정규화를 위한 영상 세로 크기

# ==========================================
# 2. 데이터 전처리 함수 (JSON -> 숫자 변환)
# ==========================================
def extract_keypoints(json_path):
    """ JSON 파일에서 몸통, 왼손, 오른손 좌표를 꺼내 0~1 사이로 정규화합니다. """
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except:
        return np.zeros(134) # 파일이 깨졌으면 0으로 채움

    # 데이터가 비어있는지 확인
    if not data.get('people'):
        return np.zeros(134) 

    # people 데이터 형식이 리스트인지 딕셔너리인지 확인
    people_data = data['people']
    person = people_data[0] if isinstance(people_data, list) else people_data

    pose = person.get('pose_keypoints_2d', [])
    lh = person.get('hand_left_keypoints_2d', [])
    rh = person.get('hand_right_keypoints_2d', [])

    # 좌표 추출 및 정규화 함수
    def process_points(points, count):
        xy = []
        if len(points) < count * 3:
            return [0.0] * (count * 2) # 데이터 부족하면 0으로 채움
        
        for i in range(0, count * 3, 3):
            x = points[i] / IMG_WIDTH
            y = points[i+1] / IMG_HEIGHT
            xy.extend([x, y])
        return xy

    # Pose(25개) + Left Hand(21개) + Right Hand(21개) = 총 67개 점 -> 134개 좌표
    feature_vector = []
    feature_vector.extend(process_points(pose, 25))
    feature_vector.extend(process_points(lh, 21))
    feature_vector.extend(process_points(rh, 21))

    return np.array(feature_vector)

# ==========================================
# 3. 데이터셋 로딩 (단어 통합 기능 포함)
# ==========================================
def load_dataset(base_path):
    X_data = []
    Y_data = []

    if not os.path.exists(base_path):
        print(f"❌ 오류: '{base_path}' 폴더가 없습니다.")
        return [], []

    folders = os.listdir(base_path)
    print(f"📂 총 {len(folders)}개의 폴더를 발견했습니다. 로딩을 시작합니다...")

    for folder_name in folders:
        folder_path = os.path.join(base_path, folder_name)
        if not os.path.isdir(folder_path):
            continue

        # 1. 정답지(morpheme.json) 찾기
        morpheme_file = None
        for file in os.listdir(folder_path):
            # 정면(_F_)이면서 morpheme 파일인 것 찾기
            if "_F_" in file and file.endswith("morpheme.json"):
                morpheme_file = file
                break
        
        if morpheme_file is None:
            print(f"⚠️ 경고: {folder_name}에 morpheme 파일이 없습니다. 건너뜁니다.")
            continue

        # 2. 파일 열어서 정보 읽기
        with open(os.path.join(folder_path, morpheme_file), 'r', encoding='utf-8') as f:
            try:
                m_data = json.load(f)
                raw_label = m_data['data'][0]['attributes'][0]['name'] # 원본 단어 이름
                
                # ========================================================
                # [중요] 단어 이름 통일하기 (사용자 맞춤 설정)
                # ========================================================
                if raw_label in ["안녕하세요", "안녕"]:
                    label = "안녕"
                elif raw_label in ["감사합니다", "감사"]:
                    label = "감사"
                elif raw_label in ["미안합니다", "미안"]:
                    label = "미안"
                elif raw_label == "나":
                    label = "나"
                elif raw_label == "좋다":
                    label = "좋다"
                else:
                    label = raw_label # 그 외는 그대로 사용
                
                # 시작/종료 시간 가져오기
                start_time = float(m_data['data'][0]['start'])
                end_time = float(m_data['data'][0]['end'])
            except Exception as e:
                print(f"⚠️ 데이터 읽기 오류 ({folder_name}): {e}")
                continue

        print(f"   Reading: [{label}] (원본: {raw_label})")

        # 3. 시간 -> 프레임 번호로 변환
        start_frame = int(start_time * 30)
        end_frame = int(end_time * 30)

        sequence = []
        
        # 4. 해당 구간의 Keypoint 파일들 읽어오기
        for frame_idx in range(start_frame, end_frame + 1):
            # 파일명 규칙: ..._F_...00052_keypoints.json
            suffix = f"_{frame_idx:012d}_keypoints.json"
            
            target_json = None
            for file in os.listdir(folder_path):
                if "_F_" in file and file.endswith(suffix):
                    target_json = file
                    break
            
            if target_json:
                kp_vector = extract_keypoints(os.path.join(folder_path, target_json))
                sequence.append(kp_vector)

        # 데이터가 잘 모였으면 리스트에 추가
        if len(sequence) > 0:
            X_data.append(sequence)
            Y_data.append(label)

    return X_data, Y_data

# ==========================================
# 4. 학습 실행
# ==========================================

# 1) 데이터 로드
X, Y = load_dataset(DATA_PATH)

if len(X) == 0:
    print("\n❌ 학습할 데이터가 없습니다! dataset 폴더 구성을 확인해주세요.")
    exit()

# 2) 데이터 가공 (패딩 & 인코딩)
X_padded = pad_sequences(X, maxlen=MAX_FRAMES, padding='post', dtype='float32')
X_array = np.array(X_padded)

encoder = LabelEncoder()
Y_encoded = encoder.fit_transform(Y)
Y_onehot = to_categorical(Y_encoded)

print("\n" + "="*50)
print(f"✅ 데이터 준비 완료!")
print(f" - 학습 데이터 수: {len(X_array)}개")
print(f" - 학습할 단어들: {encoder.classes_}")
print("="*50 + "\n")

# 3) GRU 모델 설계
model = Sequential()
model.add(GRU(64, return_sequences=True, activation='relu', input_shape=(MAX_FRAMES, 134)))
model.add(Dropout(0.2))
model.add(GRU(128, return_sequences=False, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(encoder.classes_), activation='softmax')) # 단어 개수만큼 출력

# 4) 컴파일 및 학습
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("🚀 모델 학습을 시작합니다...")
model.fit(X_array, Y_onehot, epochs=50, batch_size=2, validation_split=0.1)

# 5) 저장
model.save('my_sign_language_model.h5')
np.save('classes.npy', encoder.classes_)

print("\n🎉 학습이 성공적으로 끝났습니다! (모델 저장됨)")